In [91]:
class Perceptron:
    def __init__(self, num_inputs, activation_f, error_f):
        # self.weights = np.random.rand(num_inputs)
        # self.bias = np.random.rand(0)
        self.weights = np.zeros(num_inputs)
        self.bias = 0
        self.activation = activation_f
        self.error = error_f

    def feedforward(self, inputs):
        total = np.dot(self.weights, inputs) + self.bias
        return total
    
    def predict(self, inputs):
        return self.activation(np.dot(self.weights, inputs) + self.bias)
        
    def train(self, training_inputs, reals, learning_rate, epochs):
        for _ in range(epochs):
            for inputs, real in zip(training_inputs, reals):
                prediction = self.feedforward(inputs)
                # 0 is 1, 1 is 2
                if real == 1 and prediction > 0:
                    self.weights -= learning_rate * inputs
                if real == 0 and prediction <= 0:
                    self.weights += learning_rate * inputs


In [2]:
import numpy as np # linear algebra
import struct
from array import array
from os.path  import join

#
# MNIST Data Loader Class
#
class MnistDataloader(object):
    def __init__(self, training_images_filepath,training_labels_filepath,
                 test_images_filepath, test_labels_filepath):
        self.training_images_filepath = training_images_filepath
        self.training_labels_filepath = training_labels_filepath
        self.test_images_filepath = test_images_filepath
        self.test_labels_filepath = test_labels_filepath
    
    def read_images_labels(self, images_filepath, labels_filepath):        
        labels = []
        with open(labels_filepath, 'rb') as file:
            magic, size = struct.unpack(">II", file.read(8))
            if magic != 2049:
                raise ValueError('Magic number mismatch, expected 2049, got {}'.format(magic))
            labels = array("B", file.read())        
        
        with open(images_filepath, 'rb') as file:
            magic, size, rows, cols = struct.unpack(">IIII", file.read(16))
            if magic != 2051:
                raise ValueError('Magic number mismatch, expected 2051, got {}'.format(magic))
            image_data = array("B", file.read())        
        images = []
        for i in range(size):
            images.append([0] * rows * cols)
        for i in range(size):
            img = np.array(image_data[i * rows * cols:(i + 1) * rows * cols])
            img = img.reshape(28, 28)
            images[i][:] = img            
        
        return images, labels
            
    def load_data(self):
        x_train, y_train = self.read_images_labels(self.training_images_filepath, self.training_labels_filepath)
        x_test, y_test = self.read_images_labels(self.test_images_filepath, self.test_labels_filepath)
        return (x_train, y_train),(x_test, y_test)        

In [17]:
(x_train_raw, y_train_raw),(x_test_raw, y_test_raw) = MnistDataloader(
    "data/mnist/train-images.idx3-ubyte",
    "data/mnist/train-labels.idx1-ubyte",
    "data/mnist/t10k-images.idx3-ubyte",
    "data/mnist/t10k-labels.idx1-ubyte"
    ).load_data()

In [18]:
x_train_raw = np.array(x_train_raw).reshape(60000, 28*28)
y_train_raw = np.array(y_train_raw)
x_test_raw = np.array(x_test_raw).reshape(10000, 28*28)
y_test_raw = np.array(y_test_raw)

In [27]:
# ones = y_train == 1
# twos = y_train == 2
two_ones = ((y_train_raw == 1) | (y_train_raw == 2))
y_train = y_train_raw[two_ones]
x_train = x_train_raw[two_ones]

ones = y_train == 1
twos = y_train == 2

In [28]:
x_train_raw.shape, y_train.shape

((60000, 784), (12700,))

In [138]:
errorf = lambda p, r: (p - r)**2 / 2

p = Perceptron(28*28, lambda x: 1 if x >= 0 else -1, errorf)
# p2 = Perceptron(28*28, lambda x: 1 / (1 + np.exp(-x)), errorf)
# p1.train(x_train, ones, 0.1, 1)
p.train(x_train, twos, 8, 1)

In [116]:
two_ones = ((y_test_raw == 1) | (y_test_raw == 2))
y_test = y_test_raw[two_ones]
x_test = x_test_raw[two_ones]

ones_test = y_test == 1
twos_test = y_test == 2
x_test.shape

(2167, 784)

In [139]:
is_right = 0
pred_ones = 0
pred_twos = 0
N = 2167
for i, x in enumerate(x_test[:N, :]):
    prediction = p.predict(x)
    prediction_num = 2 if prediction < 0 else 1
    pred_ones += prediction_num == 1
    pred_twos += prediction_num == 2
    test_num = 1 if ones_test[i] else 2
    is_right += prediction_num == test_num
pred_ones / N, pred_twos / N, is_right / N

(0.5251499769266267, 0.4748500230733733, 0.9940009229349331)

In [140]:
is_right = 0
pred_ones = 0
pred_twos = 0
N = x_train.shape[0]
for i, x in enumerate(x_train[:N, :]):
    prediction = p.predict(x)
    prediction_num = 2 if prediction < 0 else 1
    pred_ones += prediction_num == 1
    pred_twos += prediction_num == 2
    test_num = 1 if ones[i] else 2
    is_right += prediction_num == test_num
pred_ones / N, pred_twos / N, is_right / N

(0.5335433070866141, 0.4664566929133858, 0.991496062992126)